In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd, datetime
import numpy as np
import requests
import time
from scipy.stats import linregress
from ydata_profiling import ProfileReport
from pathlib import Path


# Import the OpenWeatherMap API key
from api_keys import airpollution_api_key 


# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy

# airpollution_api_key = airpollution_api_key or "YOUR_API_KEY"
print(airpollution_api_key)

cffa2b2ac0605c550af5b2dd12540b15


In [2]:
# Load the CSV file into a Pandas DataFrame
city_health = pd.read_csv("../Resources/CDC_500_City_Health.csv")

# Display the data table for preview
city_health

,Year,StateAbbr,StateDesc,CityName,GeographicLevel,DataSource,Category,UniqueID,Measure,Data_Value_Unit,...,High_Confidence_Limit,Data_Value_Footnote_Symbol,Data_Value_Footnote,PopulationCount,GeoLocation,CategoryID,MeasureId,CityFIPS,TractFIPS,Short_Question_Text
0,2017,CA,California,Hayward,City,BRFSS,Health Outcomes,633000,Coronary heart disease among adults aged >=18 ...,%,...,4.8,NaN,NaN,144186,"(37.6329591551, -122.077051051)",HLTHOUT,CHD,633000,NaN,Coronary Heart Disease
1,2017,CA,California,Indio,City,BRFSS,Health Outcomes,636448,Chronic obstructive pulmonary disease among ad...,%,...,6.2,NaN,NaN,76036,"(33.7298067837, -116.237258141)",HLTHOUT,COPD,636448,NaN,COPD
2,2017,CA,California,Bellflower,City,BRFSS,Health Outcomes,604982,Coronary heart disease among adults aged >=18 ...,%,...,5.4,NaN,NaN,76616,"(33.8880417923, -118.127100236)",HLTHOUT,CHD,604982,NaN,Coronary Heart Disease
3,2017,CA,California,Lynwood,City,BRFSS,Health Outcomes,644574,Cancer (excluding skin cancer) among adults ag...,%,...,5.1,NaN,NaN,69772,"(33.9239616867, -118.201648375)",HLTHOUT,CANCER,644574,NaN,Cancer (except skin)
4,2017,CA,California,Redding,City,BRFSS,Health Outcomes,659920,Cancer (excluding skin cancer) among adults ag...,%,...,6.7,NaN,NaN,89861,"(40.5697591271, -122.365026322)",HLTHOUT,CANCER,659920,NaN,Cancer (except skin)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,2017,WI,Wisconsin,Racine,City,BRFSS,Health Outcomes,5566000,Chronic obstructive pulmonary disease among ad...,%,...,6.5,NaN,NaN,78860,"(42.7274599494, -87.813453024)",HLTHOUT,COPD,5566000,NaN,COPD
1996,2017,WI,Wisconsin,Waukesha,City,BRFSS,Health Outcomes,5584250,Cancer (excluding skin cancer) among adults ag...,%,...,6.5,NaN,NaN,70718,"(43.0093332215, -88.2457679157)",HLTHOUT,CANCER,5584250,NaN,Cancer (except skin)
1997,2017,WI,Wisconsin,Racine,City,BRFSS,Health Outcomes,5566000,Cancer (excluding skin cancer) among adults ag...,%,...,6.3,NaN,NaN,78860,"(42.7274599494, -87.813453024)",HLTHOUT,CANCER,5566000,NaN,Cancer (except skin)
1998,2017,WI,Wisconsin,Racine,City,BRFSS,Health Outcomes,5566000,Coronary heart disease among adults aged >=18 ...,%,...,6.1,NaN,NaN,78860,"(42.7274599494, -87.813453024)",HLTHOUT,CHD,5566000,NaN,Coronary Heart Disease


In [3]:
# review data types
city_health.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Year                        2000 non-null   int64  
 1   StateAbbr                   2000 non-null   object 
 2   StateDesc                   2000 non-null   object 
 3   CityName                    2000 non-null   object 
 4   GeographicLevel             2000 non-null   object 
 5   DataSource                  2000 non-null   object 
 6   Category                    2000 non-null   object 
 7   UniqueID                    2000 non-null   int64  
 8   Measure                     2000 non-null   object 
 9   Data_Value_Unit             2000 non-null   object 
 10  DataValueTypeID             2000 non-null   object 
 11  Data_Value_Type             2000 non-null   object 
 12  Data_Value                  2000 non-null   float64
 13  Low_Confidence_Limit        2000 

In [4]:
# describe data
city_health.describe()

,Year,UniqueID,Data_Value,Low_Confidence_Limit,High_Confidence_Limit,Data_Value_Footnote_Symbol,Data_Value_Footnote,PopulationCount,CityFIPS,TractFIPS
count,2000.0,2.000000e+03,2000.00000,2000.000000,2000.000000,0.0,0.0,2.000000e+03,2.000000e+03,0.0
mean,2017.0,2.438986e+06,6.86405,6.758650,6.973250,NaN,NaN,2.060416e+05,2.438986e+06,NaN
std,0.0,1.719003e+06,1.89987,1.891852,1.908738,NaN,NaN,4.571376e+05,1.719003e+06,NaN
min,2017.0,1.500300e+04,3.10000,2.900000,3.200000,NaN,NaN,4.241700e+04,1.500300e+04,NaN
25%,2017.0,6.700735e+05,5.60000,5.500000,5.700000,NaN,NaN,8.159000e+04,6.700735e+05,NaN
50%,2017.0,1.955965e+06,6.30000,6.300000,6.400000,NaN,NaN,1.061060e+05,1.955965e+06,NaN
75%,2017.0,4.053125e+06,8.20000,8.100000,8.400000,NaN,NaN,1.811278e+05,4.053125e+06,NaN
max,2017.0,5.613900e+06,14.20000,14.100000,14.300000,NaN,NaN,8.175133e+06,5.613900e+06,NaN


In [5]:
# Run the Profiling Report using ydata_profiling
# profile = ProfileReport(city_health,title = "City Health Profile Report")
# profile.to_notebook_iframe()

In [6]:
# drop columns with null values
clean_city_health = city_health.drop(['Data_Value_Footnote_Symbol','Data_Value_Footnote','TractFIPS'],axis=1)

In [7]:
# review data types after dropping null values
clean_city_health.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Year                   2000 non-null   int64  
 1   StateAbbr              2000 non-null   object 
 2   StateDesc              2000 non-null   object 
 3   CityName               2000 non-null   object 
 4   GeographicLevel        2000 non-null   object 
 5   DataSource             2000 non-null   object 
 6   Category               2000 non-null   object 
 7   UniqueID               2000 non-null   int64  
 8   Measure                2000 non-null   object 
 9   Data_Value_Unit        2000 non-null   object 
 10  DataValueTypeID        2000 non-null   object 
 11  Data_Value_Type        2000 non-null   object 
 12  Data_Value             2000 non-null   float64
 13  Low_Confidence_Limit   2000 non-null   float64
 14  High_Confidence_Limit  2000 non-null   float64
 15  Popu

In [8]:
# describe data after dropping null values
clean_city_health.describe()

,Year,UniqueID,Data_Value,Low_Confidence_Limit,High_Confidence_Limit,PopulationCount,CityFIPS
count,2000.0,2.000000e+03,2000.00000,2000.000000,2000.000000,2.000000e+03,2.000000e+03
mean,2017.0,2.438986e+06,6.86405,6.758650,6.973250,2.060416e+05,2.438986e+06
std,0.0,1.719003e+06,1.89987,1.891852,1.908738,4.571376e+05,1.719003e+06
min,2017.0,1.500300e+04,3.10000,2.900000,3.200000,4.241700e+04,1.500300e+04
25%,2017.0,6.700735e+05,5.60000,5.500000,5.700000,8.159000e+04,6.700735e+05
50%,2017.0,1.955965e+06,6.30000,6.300000,6.400000,1.061060e+05,1.955965e+06
75%,2017.0,4.053125e+06,8.20000,8.100000,8.400000,1.811278e+05,4.053125e+06
max,2017.0,5.613900e+06,14.20000,14.100000,14.300000,8.175133e+06,5.613900e+06


In [9]:
# Run the Profiling Report using ydata_profiling after dropping null values
# profile = ProfileReport(clean_city_health,title = "City Health Profile Report")
# profile.to_notebook_iframe()

In [10]:
# Review the number of cities per state per unique MeasureID
diseases_per_state = clean_city_health.groupby('StateDesc')['MeasureId'].value_counts()


diseases_per_state


StateDesc  MeasureId
Alabama    CANCER       6
           CASTHMA      6
           CHD          6
           COPD         6
Alaska     CANCER       1
                       ..
Wisconsin  COPD         7
Wyoming    CANCER       1
           CASTHMA      1
           CHD          1
           COPD         1
Name: count, Length: 204, dtype: int64

In [11]:
# total count of diseases per state
diseases_per_state_total = clean_city_health.groupby('StateDesc')['MeasureId'].count().sort_values(ascending=False)
diseases_per_state_total



StateDesc
California       484
Texas            188
Florida          132
Illinois          72
Michigan          64
Colorado          56
Washington        56
North Carolin     56
Massachusetts     52
Arizona           48
Indiana           44
Virginia          44
Georgia           44
Ohio              36
New York          36
New Jersey        36
Utah              36
Oregon            32
Connecticut       32
Missouri          32
Pennsylvania      28
Wisconsin         28
Minnesota         28
Tennessee         24
Oklahoma          24
Alabama           24
Kansas            24
Iowa              24
Louisiana         24
Nevada            20
Arkansas          20
South Carolin     20
New Mexico        16
Rhode Island      16
Idaho             12
Nebraska           8
South Dakota       8
Montana            8
Kentucky           8
Mississippi        8
New Hampshire      8
Maine              4
North Dakota       4
Maryland           4
Alaska             4
Hawaii             4
Vermont            4
Dis

In [12]:
# number of cities per state with diseases
cities_per_state = clean_city_health.groupby('StateDesc')['CityName'].nunique().sort_values(ascending=False)
cities_per_state

StateDesc
California       121
Texas             47
Florida           33
Illinois          18
Michigan          16
Colorado          14
Washington        14
North Carolin     14
Massachusetts     13
Arizona           12
Indiana           11
Virginia          11
Georgia           11
Ohio               9
New York           9
New Jersey         9
Utah               9
Oregon             8
Connecticut        8
Missouri           8
Pennsylvania       7
Wisconsin          7
Minnesota          7
Tennessee          6
Oklahoma           6
Alabama            6
Kansas             6
Iowa               6
Louisiana          6
Nevada             5
Arkansas           5
South Carolin      5
New Mexico         4
Rhode Island       4
Idaho              3
Nebraska           2
South Dakota       2
Montana            2
Kentucky           2
Mississippi        2
New Hampshire      2
Maine              1
North Dakota       1
Maryland           1
Alaska             1
Hawaii             1
Vermont            1
Dis

In [13]:
clean_city_health[['Latitude', 'Longitude']] = clean_city_health['GeoLocation'].str.extract(r'\(([^,]+),([^)]+)\)', expand=True)
clean_city_health['Latitude'] = clean_city_health['Latitude'].astype(float)
clean_city_health['Longitude'] = clean_city_health['Longitude'].astype(float)
clean_city_health

,Year,StateAbbr,StateDesc,CityName,GeographicLevel,DataSource,Category,UniqueID,Measure,Data_Value_Unit,...,Low_Confidence_Limit,High_Confidence_Limit,PopulationCount,GeoLocation,CategoryID,MeasureId,CityFIPS,Short_Question_Text,Latitude,Longitude
0,2017,CA,California,Hayward,City,BRFSS,Health Outcomes,633000,Coronary heart disease among adults aged >=18 ...,%,...,4.7,4.8,144186,"(37.6329591551, -122.077051051)",HLTHOUT,CHD,633000,Coronary Heart Disease,37.632959,-122.077051
1,2017,CA,California,Indio,City,BRFSS,Health Outcomes,636448,Chronic obstructive pulmonary disease among ad...,%,...,5.8,6.2,76036,"(33.7298067837, -116.237258141)",HLTHOUT,COPD,636448,COPD,33.729807,-116.237258
2,2017,CA,California,Bellflower,City,BRFSS,Health Outcomes,604982,Coronary heart disease among adults aged >=18 ...,%,...,5.2,5.4,76616,"(33.8880417923, -118.127100236)",HLTHOUT,CHD,604982,Coronary Heart Disease,33.888042,-118.127100
3,2017,CA,California,Lynwood,City,BRFSS,Health Outcomes,644574,Cancer (excluding skin cancer) among adults ag...,%,...,5.0,5.1,69772,"(33.9239616867, -118.201648375)",HLTHOUT,CANCER,644574,Cancer (except skin),33.923962,-118.201648
4,2017,CA,California,Redding,City,BRFSS,Health Outcomes,659920,Cancer (excluding skin cancer) among adults ag...,%,...,6.6,6.7,89861,"(40.5697591271, -122.365026322)",HLTHOUT,CANCER,659920,Cancer (except skin),40.569759,-122.365026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,2017,WI,Wisconsin,Racine,City,BRFSS,Health Outcomes,5566000,Chronic obstructive pulmonary disease among ad...,%,...,6.1,6.5,78860,"(42.7274599494, -87.813453024)",HLTHOUT,COPD,5566000,COPD,42.727460,-87.813453
1996,2017,WI,Wisconsin,Waukesha,City,BRFSS,Health Outcomes,5584250,Cancer (excluding skin cancer) among adults ag...,%,...,6.4,6.5,70718,"(43.0093332215, -88.2457679157)",HLTHOUT,CANCER,5584250,Cancer (except skin),43.009333,-88.245768
1997,2017,WI,Wisconsin,Racine,City,BRFSS,Health Outcomes,5566000,Cancer (excluding skin cancer) among adults ag...,%,...,6.2,6.3,78860,"(42.7274599494, -87.813453024)",HLTHOUT,CANCER,5566000,Cancer (except skin),42.727460,-87.813453
1998,2017,WI,Wisconsin,Racine,City,BRFSS,Health Outcomes,5566000,Coronary heart disease among adults aged >=18 ...,%,...,5.9,6.1,78860,"(42.7274599494, -87.813453024)",HLTHOUT,CHD,5566000,Coronary Heart Disease,42.727460,-87.813453


In [14]:
clean_city_health = clean_city_health.sort_values(by='CityName')
clean_city_health

,Year,StateAbbr,StateDesc,CityName,GeographicLevel,DataSource,Category,UniqueID,Measure,Data_Value_Unit,...,Low_Confidence_Limit,High_Confidence_Limit,PopulationCount,GeoLocation,CategoryID,MeasureId,CityFIPS,Short_Question_Text,Latitude,Longitude
1634,2017,TX,Texas,Abilene,City,BRFSS,Health Outcomes,4801000,Cancer (excluding skin cancer) among adults ag...,%,...,6.1,6.2,117063,"(32.4546510898, -99.7383043723)",HLTHOUT,CANCER,4801000,Cancer (except skin),32.454651,-99.738304
1644,2017,TX,Texas,Abilene,City,BRFSS,Health Outcomes,4801000,Chronic obstructive pulmonary disease among ad...,%,...,6.5,6.9,117063,"(32.4546510898, -99.7383043723)",HLTHOUT,COPD,4801000,COPD,32.454651,-99.738304
1645,2017,TX,Texas,Abilene,City,BRFSS,Health Outcomes,4801000,Current asthma among adults aged >=18 Years,%,...,8.7,8.8,117063,"(32.4546510898, -99.7383043723)",HLTHOUT,CASTHMA,4801000,Current Asthma,32.454651,-99.738304
1659,2017,TX,Texas,Abilene,City,BRFSS,Health Outcomes,4801000,Coronary heart disease among adults aged >=18 ...,%,...,6.2,6.4,117063,"(32.4546510898, -99.7383043723)",HLTHOUT,CHD,4801000,Coronary Heart Disease,32.454651,-99.738304
1468,2017,OH,Ohio,Akron,City,BRFSS,Health Outcomes,3901000,Coronary heart disease among adults aged >=18 ...,%,...,7.4,7.5,199110,"(41.0801444942, -81.5219213688)",HLTHOUT,CHD,3901000,Coronary Heart Disease,41.080144,-81.521921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1509,2017,OH,Ohio,Youngstown,City,BRFSS,Health Outcomes,3988000,Chronic obstructive pulmonary disease among ad...,%,...,10.9,11.4,66982,"(41.0993286607, -80.6463187081)",HLTHOUT,COPD,3988000,COPD,41.099329,-80.646319
98,2017,AZ,Arizona,Yuma,City,BRFSS,Health Outcomes,485540,Coronary heart disease among adults aged >=18 ...,%,...,5.8,6.1,93064,"(32.5986027117, -114.54836837)",HLTHOUT,CHD,485540,Coronary Heart Disease,32.598603,-114.548368
91,2017,AZ,Arizona,Yuma,City,BRFSS,Health Outcomes,485540,Current asthma among adults aged >=18 Years,%,...,9.7,9.9,93064,"(32.5986027117, -114.54836837)",HLTHOUT,CASTHMA,485540,Current Asthma,32.598603,-114.548368
70,2017,AZ,Arizona,Yuma,City,BRFSS,Health Outcomes,485540,Chronic obstructive pulmonary disease among ad...,%,...,6.5,6.9,93064,"(32.5986027117, -114.54836837)",HLTHOUT,COPD,485540,COPD,32.598603,-114.548368


In [15]:
clean_city_health.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2000 entries, 1634 to 62
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Year                   2000 non-null   int64  
 1   StateAbbr              2000 non-null   object 
 2   StateDesc              2000 non-null   object 
 3   CityName               2000 non-null   object 
 4   GeographicLevel        2000 non-null   object 
 5   DataSource             2000 non-null   object 
 6   Category               2000 non-null   object 
 7   UniqueID               2000 non-null   int64  
 8   Measure                2000 non-null   object 
 9   Data_Value_Unit        2000 non-null   object 
 10  DataValueTypeID        2000 non-null   object 
 11  Data_Value_Type        2000 non-null   object 
 12  Data_Value             2000 non-null   float64
 13  Low_Confidence_Limit   2000 non-null   float64
 14  High_Confidence_Limit  2000 non-null   float64
 15  Populati

In [16]:
# create a dataframe for cancer data eliminating the index
cancer_df = clean_city_health.loc[clean_city_health['MeasureId'] == 'CANCER']
cancer_df.reset_index(drop=True, inplace=True)

cancer_df




,Year,StateAbbr,StateDesc,CityName,GeographicLevel,DataSource,Category,UniqueID,Measure,Data_Value_Unit,...,Low_Confidence_Limit,High_Confidence_Limit,PopulationCount,GeoLocation,CategoryID,MeasureId,CityFIPS,Short_Question_Text,Latitude,Longitude
0,2017,TX,Texas,Abilene,City,BRFSS,Health Outcomes,4801000,Cancer (excluding skin cancer) among adults ag...,%,...,6.1,6.2,117063,"(32.4546510898, -99.7383043723)",HLTHOUT,CANCER,4801000,Cancer (except skin),32.454651,-99.738304
1,2017,OH,Ohio,Akron,City,BRFSS,Health Outcomes,3901000,Cancer (excluding skin cancer) among adults ag...,%,...,6.1,6.2,199110,"(41.0801444942, -81.5219213688)",HLTHOUT,CANCER,3901000,Cancer (except skin),41.080144,-81.521921
2,2017,CA,California,Alameda,City,BRFSS,Health Outcomes,600562,Cancer (excluding skin cancer) among adults ag...,%,...,5.7,5.8,73812,"(37.7650849031, -122.266489842)",HLTHOUT,CANCER,600562,Cancer (except skin),37.765085,-122.266490
3,2017,GA,Georgia,Albany,City,BRFSS,Health Outcomes,1301052,Cancer (excluding skin cancer) among adults ag...,%,...,5.9,6.0,77434,"(31.5776036766, -84.1763240815)",HLTHOUT,CANCER,1301052,Cancer (except skin),31.577604,-84.176324
4,2017,NY,New York,Albany,City,BRFSS,Health Outcomes,3601000,Cancer (excluding skin cancer) among adults ag...,%,...,6.1,6.2,97856,"(42.6663969604, -73.7986826267)",HLTHOUT,CANCER,3601000,Cancer (except skin),42.666397,-73.798683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2017,MI,Michigan,Wyoming,City,BRFSS,Health Outcomes,2688940,Cancer (excluding skin cancer) among adults ag...,%,...,6.3,6.5,72125,"(42.8909745661, -85.7067229249)",HLTHOUT,CANCER,2688940,Cancer (except skin),42.890975,-85.706723
496,2017,WA,Washington,Yakima,City,BRFSS,Health Outcomes,5380010,Cancer (excluding skin cancer) among adults ag...,%,...,6.1,6.2,91067,"(46.5925792092, -120.547807614)",HLTHOUT,CANCER,5380010,Cancer (except skin),46.592579,-120.547808
497,2017,NY,New York,Yonkers,City,BRFSS,Health Outcomes,3684000,Cancer (excluding skin cancer) among adults ag...,%,...,5.7,5.8,195976,"(40.9465025597, -73.8636959965)",HLTHOUT,CANCER,3684000,Cancer (except skin),40.946503,-73.863696
498,2017,OH,Ohio,Youngstown,City,BRFSS,Health Outcomes,3988000,Cancer (excluding skin cancer) among adults ag...,%,...,5.9,6.0,66982,"(41.0993286607, -80.6463187081)",HLTHOUT,CANCER,3988000,Cancer (except skin),41.099329,-80.646319


In [17]:
# Create a dataframe for copd data eliminating the index
copd_df = clean_city_health.loc[clean_city_health['MeasureId'] == 'COPD']
copd_df.reset_index(drop=True, inplace=True)

copd_df

,Year,StateAbbr,StateDesc,CityName,GeographicLevel,DataSource,Category,UniqueID,Measure,Data_Value_Unit,...,Low_Confidence_Limit,High_Confidence_Limit,PopulationCount,GeoLocation,CategoryID,MeasureId,CityFIPS,Short_Question_Text,Latitude,Longitude
0,2017,TX,Texas,Abilene,City,BRFSS,Health Outcomes,4801000,Chronic obstructive pulmonary disease among ad...,%,...,6.5,6.9,117063,"(32.4546510898, -99.7383043723)",HLTHOUT,COPD,4801000,COPD,32.454651,-99.738304
1,2017,OH,Ohio,Akron,City,BRFSS,Health Outcomes,3901000,Chronic obstructive pulmonary disease among ad...,%,...,9.2,9.5,199110,"(41.0801444942, -81.5219213688)",HLTHOUT,COPD,3901000,COPD,41.080144,-81.521921
2,2017,CA,California,Alameda,City,BRFSS,Health Outcomes,600562,Chronic obstructive pulmonary disease among ad...,%,...,3.8,4.1,73812,"(37.7650849031, -122.266489842)",HLTHOUT,COPD,600562,COPD,37.765085,-122.266490
3,2017,GA,Georgia,Albany,City,BRFSS,Health Outcomes,1301052,Chronic obstructive pulmonary disease among ad...,%,...,8.7,9.3,77434,"(31.5776036766, -84.1763240815)",HLTHOUT,COPD,1301052,COPD,31.577604,-84.176324
4,2017,NY,New York,Albany,City,BRFSS,Health Outcomes,3601000,Chronic obstructive pulmonary disease among ad...,%,...,6.6,6.9,97856,"(42.6663969604, -73.7986826267)",HLTHOUT,COPD,3601000,COPD,42.666397,-73.798683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2017,MI,Michigan,Wyoming,City,BRFSS,Health Outcomes,2688940,Chronic obstructive pulmonary disease among ad...,%,...,7.6,8.1,72125,"(42.8909745661, -85.7067229249)",HLTHOUT,COPD,2688940,COPD,42.890975,-85.706723
496,2017,WA,Washington,Yakima,City,BRFSS,Health Outcomes,5380010,Chronic obstructive pulmonary disease among ad...,%,...,7.3,7.8,91067,"(46.5925792092, -120.547807614)",HLTHOUT,COPD,5380010,COPD,46.592579,-120.547808
497,2017,NY,New York,Yonkers,City,BRFSS,Health Outcomes,3684000,Chronic obstructive pulmonary disease among ad...,%,...,5.7,5.9,195976,"(40.9465025597, -73.8636959965)",HLTHOUT,COPD,3684000,COPD,40.946503,-73.863696
498,2017,OH,Ohio,Youngstown,City,BRFSS,Health Outcomes,3988000,Chronic obstructive pulmonary disease among ad...,%,...,10.9,11.4,66982,"(41.0993286607, -80.6463187081)",HLTHOUT,COPD,3988000,COPD,41.099329,-80.646319


In [18]:
# create a dataframe for asthma data eliminating the index
casthma_df = clean_city_health.loc[clean_city_health['MeasureId'] == 'CASTHMA']
casthma_df.reset_index(drop=True, inplace=True)
casthma_df 

,Year,StateAbbr,StateDesc,CityName,GeographicLevel,DataSource,Category,UniqueID,Measure,Data_Value_Unit,...,Low_Confidence_Limit,High_Confidence_Limit,PopulationCount,GeoLocation,CategoryID,MeasureId,CityFIPS,Short_Question_Text,Latitude,Longitude
0,2017,TX,Texas,Abilene,City,BRFSS,Health Outcomes,4801000,Current asthma among adults aged >=18 Years,%,...,8.7,8.8,117063,"(32.4546510898, -99.7383043723)",HLTHOUT,CASTHMA,4801000,Current Asthma,32.454651,-99.738304
1,2017,OH,Ohio,Akron,City,BRFSS,Health Outcomes,3901000,Current asthma among adults aged >=18 Years,%,...,10.7,10.9,199110,"(41.0801444942, -81.5219213688)",HLTHOUT,CASTHMA,3901000,Current Asthma,41.080144,-81.521921
2,2017,CA,California,Alameda,City,BRFSS,Health Outcomes,600562,Current asthma among adults aged >=18 Years,%,...,8.3,8.5,73812,"(37.7650849031, -122.266489842)",HLTHOUT,CASTHMA,600562,Current Asthma,37.765085,-122.266490
3,2017,GA,Georgia,Albany,City,BRFSS,Health Outcomes,1301052,Current asthma among adults aged >=18 Years,%,...,10.9,11.3,77434,"(31.5776036766, -84.1763240815)",HLTHOUT,CASTHMA,1301052,Current Asthma,31.577604,-84.176324
4,2017,NY,New York,Albany,City,BRFSS,Health Outcomes,3601000,Current asthma among adults aged >=18 Years,%,...,10.5,10.8,97856,"(42.6663969604, -73.7986826267)",HLTHOUT,CASTHMA,3601000,Current Asthma,42.666397,-73.798683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2017,MI,Michigan,Wyoming,City,BRFSS,Health Outcomes,2688940,Current asthma among adults aged >=18 Years,%,...,10.2,10.6,72125,"(42.8909745661, -85.7067229249)",HLTHOUT,CASTHMA,2688940,Current Asthma,42.890975,-85.706723
496,2017,WA,Washington,Yakima,City,BRFSS,Health Outcomes,5380010,Current asthma among adults aged >=18 Years,%,...,11.2,11.5,91067,"(46.5925792092, -120.547807614)",HLTHOUT,CASTHMA,5380010,Current Asthma,46.592579,-120.547808
497,2017,NY,New York,Yonkers,City,BRFSS,Health Outcomes,3684000,Current asthma among adults aged >=18 Years,%,...,9.7,9.9,195976,"(40.9465025597, -73.8636959965)",HLTHOUT,CASTHMA,3684000,Current Asthma,40.946503,-73.863696
498,2017,OH,Ohio,Youngstown,City,BRFSS,Health Outcomes,3988000,Current asthma among adults aged >=18 Years,%,...,11.4,11.6,66982,"(41.0993286607, -80.6463187081)",HLTHOUT,CASTHMA,3988000,Current Asthma,41.099329,-80.646319


In [19]:
# create a dataframe for chd data eliminating the index
chd_df = clean_city_health.loc[clean_city_health['MeasureId'] == 'CHD']
chd_df.reset_index(drop=True, inplace=True)

chd_df


,Year,StateAbbr,StateDesc,CityName,GeographicLevel,DataSource,Category,UniqueID,Measure,Data_Value_Unit,...,Low_Confidence_Limit,High_Confidence_Limit,PopulationCount,GeoLocation,CategoryID,MeasureId,CityFIPS,Short_Question_Text,Latitude,Longitude
0,2017,TX,Texas,Abilene,City,BRFSS,Health Outcomes,4801000,Coronary heart disease among adults aged >=18 ...,%,...,6.2,6.4,117063,"(32.4546510898, -99.7383043723)",HLTHOUT,CHD,4801000,Coronary Heart Disease,32.454651,-99.738304
1,2017,OH,Ohio,Akron,City,BRFSS,Health Outcomes,3901000,Coronary heart disease among adults aged >=18 ...,%,...,7.4,7.5,199110,"(41.0801444942, -81.5219213688)",HLTHOUT,CHD,3901000,Coronary Heart Disease,41.080144,-81.521921
2,2017,CA,California,Alameda,City,BRFSS,Health Outcomes,600562,Coronary heart disease among adults aged >=18 ...,%,...,3.9,4.1,73812,"(37.7650849031, -122.266489842)",HLTHOUT,CHD,600562,Coronary Heart Disease,37.765085,-122.266490
3,2017,GA,Georgia,Albany,City,BRFSS,Health Outcomes,1301052,Coronary heart disease among adults aged >=18 ...,%,...,7.2,7.4,77434,"(31.5776036766, -84.1763240815)",HLTHOUT,CHD,1301052,Coronary Heart Disease,31.577604,-84.176324
4,2017,NY,New York,Albany,City,BRFSS,Health Outcomes,3601000,Coronary heart disease among adults aged >=18 ...,%,...,5.8,6.1,97856,"(42.6663969604, -73.7986826267)",HLTHOUT,CHD,3601000,Coronary Heart Disease,42.666397,-73.798683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2017,MI,Michigan,Wyoming,City,BRFSS,Health Outcomes,2688940,Coronary heart disease among adults aged >=18 ...,%,...,6.1,6.4,72125,"(42.8909745661, -85.7067229249)",HLTHOUT,CHD,2688940,Coronary Heart Disease,42.890975,-85.706723
496,2017,WA,Washington,Yakima,City,BRFSS,Health Outcomes,5380010,Coronary heart disease among adults aged >=18 ...,%,...,6.4,6.7,91067,"(46.5925792092, -120.547807614)",HLTHOUT,CHD,5380010,Coronary Heart Disease,46.592579,-120.547808
497,2017,NY,New York,Yonkers,City,BRFSS,Health Outcomes,3684000,Coronary heart disease among adults aged >=18 ...,%,...,5.4,5.6,195976,"(40.9465025597, -73.8636959965)",HLTHOUT,CHD,3684000,Coronary Heart Disease,40.946503,-73.863696
498,2017,OH,Ohio,Youngstown,City,BRFSS,Health Outcomes,3988000,Coronary heart disease among adults aged >=18 ...,%,...,8.6,8.9,66982,"(41.0993286607, -80.6463187081)",HLTHOUT,CHD,3988000,Coronary Heart Disease,41.099329,-80.646319


In [20]:
city_coordinates = clean_city_health['GeoLocation'].drop_duplicates()
# TODO: Delete the next line when ready to send to classmates. This is just to reduce the size of the data file.
# city_coordinates = city_coordinates.head(10)
city_coordinates

1634    (32.4546510898, -99.7383043723)
1468    (41.0801444942, -81.5219213688)
104     (37.7650849031, -122.266489842)
792     (31.5776036766, -84.1763240815)
1363    (42.6663969604, -73.7986826267)
                     ...               
1170    (42.8909745661, -85.7067229249)
1999    (46.5925792092, -120.547807614)
1384    (40.9465025597, -73.8636959965)
1484    (41.0993286607, -80.6463187081)
98       (32.5986027117, -114.54836837)
Name: GeoLocation, Length: 500, dtype: object

In [21]:
all_cities_air_pollution = []
for i in city_coordinates:
    clean_lat_lng = i.replace('(','').replace(')','').split(',')
    lon = clean_lat_lng[1].strip()
    # get data api for each lat and lng
    url = f'http://api.openweathermap.org/data/2.5/air_pollution?lat={clean_lat_lng[0]}&lon={lon}&appid={airpollution_api_key}'
    
    cityap_data = requests.get(url).json()
    coordinates = cityap_data['coord']
    components = cityap_data['list'][0]['components']
    all_cities_air_pollution.append([coordinates['lat'], coordinates['lon'], components['co'], components['no'], components['no2'], components['o3'], components['so2'], components['pm2_5'], components['pm10'], components['nh3']])


all_cities_air_pollution

[[32.4547, -99.7383, 213.62, 0.03, 1.8, 75.1, 0.33, 5.45, 10.6, 1.74],
 [41.0801, -81.5219, 247, 0, 4.71, 88.69, 1.09, 7.38, 8.31, 0.98],
 [37.77, -122.27, 247, 0.19, 3.9, 98.71, 1.04, 0.54, 0.78, 0.35],
 [31.5776, -84.1763, 230.31, 0, 2.91, 80.82, 0.36, 5.88, 8.28, 2.34],
 [42.6664, -73.7987, 310.42, 0, 16.97, 42.56, 1.18, 3.78, 4.56, 0.27],
 [35.1055, -106.6474, 198.6, 0.09, 1.65, 110.15, 0.16, 2.3, 6.52, 1.08],
 [38.8187, -77.0868, 250.34, 0, 6.51, 92.98, 1.74, 5.41, 6.82, 1.47],
 [34.08, -118.14, 253.68, 1.16, 11.65, 85.83, 1.79, 1.44, 2.74, 0.43],
 [33.1087, -96.6735, 243.66, 0.08, 8.48, 85.12, 1.46, 5.46, 11.53, 1.5],
 [40.5961, -75.4755, 263.69, 0, 4.67, 104.43, 2.77, 6.03, 6.81, 2.53],
 [35.2004, -101.8283, 198.6, 0.03, 1.2, 103, 0.12, 2.22, 4.16, 3.17],
 [33.8389, -117.8576, 260.35, 1.23, 11.82, 98.71, 3.1, 1.45, 2.87, 0.67],
 [61.1499, -149.1111, 243.66, 0.61, 1.03, 87.98, 0.07, 1.28, 1.44, 0.06],
 [42.2757, -83.7311, 233.65, 0, 3.34, 100.14, 0.51, 3.64, 4.1, 1.84],
 [37.9777

In [22]:
all_cities_air_pollution_df = pd.DataFrame(all_cities_air_pollution, columns=['Latitude', 'Longitude', 'CO', 'NO', 'NO2', 'O3', 'SO2', 'PM2.5', 'PM10', 'NH3'])
all_cities_air_pollution_df

,Latitude,Longitude,CO,NO,NO2,O3,SO2,PM2.5,PM10,NH3
0,32.4547,-99.7383,213.62,0.03,1.80,75.10,0.33,5.45,10.60,1.74
1,41.0801,-81.5219,247.00,0.00,4.71,88.69,1.09,7.38,8.31,0.98
2,37.7700,-122.2700,247.00,0.19,3.90,98.71,1.04,0.54,0.78,0.35
3,31.5776,-84.1763,230.31,0.00,2.91,80.82,0.36,5.88,8.28,2.34
4,42.6664,-73.7987,310.42,0.00,16.97,42.56,1.18,3.78,4.56,0.27
...,...,...,...,...,...,...,...,...,...,...
495,42.8910,-85.7067,280.38,0.00,8.31,80.11,0.41,5.63,7.18,6.14
496,46.5926,-120.5478,211.95,0.07,0.81,117.30,0.60,4.04,4.58,0.81
497,40.9465,-73.8637,293.73,0.00,23.99,90.84,5.60,6.75,9.18,2.19
498,41.0993,-80.6463,250.34,0.00,4.28,92.98,1.30,6.70,6.93,0.67


In [23]:

all_cities_air_pollution_df.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Latitude   500 non-null    float64
 1   Longitude  500 non-null    float64
 2   CO         500 non-null    float64
 3   NO         500 non-null    float64
 4   NO2        500 non-null    float64
 5   O3         500 non-null    float64
 6   SO2        500 non-null    float64
 7   PM2.5      500 non-null    float64
 8   PM10       500 non-null    float64
 9   NH3        500 non-null    float64
dtypes: float64(10)
memory usage: 39.2 KB


In [24]:
# merge the cancer data with the air pollution data
cancer_air_pollution_df  = pd.concat([cancer_df, all_cities_air_pollution_df], axis=1)
cancer_air_pollution_df 

,Year,StateAbbr,StateDesc,CityName,GeographicLevel,DataSource,Category,UniqueID,Measure,Data_Value_Unit,...,Latitude,Longitude,CO,NO,NO2,O3,SO2,PM2.5,PM10,NH3
0,2017,TX,Texas,Abilene,City,BRFSS,Health Outcomes,4801000,Cancer (excluding skin cancer) among adults ag...,%,...,32.4547,-99.7383,213.62,0.03,1.80,75.10,0.33,5.45,10.60,1.74
1,2017,OH,Ohio,Akron,City,BRFSS,Health Outcomes,3901000,Cancer (excluding skin cancer) among adults ag...,%,...,41.0801,-81.5219,247.00,0.00,4.71,88.69,1.09,7.38,8.31,0.98
2,2017,CA,California,Alameda,City,BRFSS,Health Outcomes,600562,Cancer (excluding skin cancer) among adults ag...,%,...,37.7700,-122.2700,247.00,0.19,3.90,98.71,1.04,0.54,0.78,0.35
3,2017,GA,Georgia,Albany,City,BRFSS,Health Outcomes,1301052,Cancer (excluding skin cancer) among adults ag...,%,...,31.5776,-84.1763,230.31,0.00,2.91,80.82,0.36,5.88,8.28,2.34
4,2017,NY,New York,Albany,City,BRFSS,Health Outcomes,3601000,Cancer (excluding skin cancer) among adults ag...,%,...,42.6664,-73.7987,310.42,0.00,16.97,42.56,1.18,3.78,4.56,0.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2017,MI,Michigan,Wyoming,City,BRFSS,Health Outcomes,2688940,Cancer (excluding skin cancer) among adults ag...,%,...,42.8910,-85.7067,280.38,0.00,8.31,80.11,0.41,5.63,7.18,6.14
496,2017,WA,Washington,Yakima,City,BRFSS,Health Outcomes,5380010,Cancer (excluding skin cancer) among adults ag...,%,...,46.5926,-120.5478,211.95,0.07,0.81,117.30,0.60,4.04,4.58,0.81
497,2017,NY,New York,Yonkers,City,BRFSS,Health Outcomes,3684000,Cancer (excluding skin cancer) among adults ag...,%,...,40.9465,-73.8637,293.73,0.00,23.99,90.84,5.60,6.75,9.18,2.19
498,2017,OH,Ohio,Youngstown,City,BRFSS,Health Outcomes,3988000,Cancer (excluding skin cancer) among adults ag...,%,...,41.0993,-80.6463,250.34,0.00,4.28,92.98,1.30,6.70,6.93,0.67


In [25]:
# merge the copd data with the air pollution data
copd_air_pollution_df  = pd.concat([copd_df, all_cities_air_pollution_df], axis=1)
copd_air_pollution_df 

,Year,StateAbbr,StateDesc,CityName,GeographicLevel,DataSource,Category,UniqueID,Measure,Data_Value_Unit,...,Latitude,Longitude,CO,NO,NO2,O3,SO2,PM2.5,PM10,NH3
0,2017,TX,Texas,Abilene,City,BRFSS,Health Outcomes,4801000,Chronic obstructive pulmonary disease among ad...,%,...,32.4547,-99.7383,213.62,0.03,1.80,75.10,0.33,5.45,10.60,1.74
1,2017,OH,Ohio,Akron,City,BRFSS,Health Outcomes,3901000,Chronic obstructive pulmonary disease among ad...,%,...,41.0801,-81.5219,247.00,0.00,4.71,88.69,1.09,7.38,8.31,0.98
2,2017,CA,California,Alameda,City,BRFSS,Health Outcomes,600562,Chronic obstructive pulmonary disease among ad...,%,...,37.7700,-122.2700,247.00,0.19,3.90,98.71,1.04,0.54,0.78,0.35
3,2017,GA,Georgia,Albany,City,BRFSS,Health Outcomes,1301052,Chronic obstructive pulmonary disease among ad...,%,...,31.5776,-84.1763,230.31,0.00,2.91,80.82,0.36,5.88,8.28,2.34
4,2017,NY,New York,Albany,City,BRFSS,Health Outcomes,3601000,Chronic obstructive pulmonary disease among ad...,%,...,42.6664,-73.7987,310.42,0.00,16.97,42.56,1.18,3.78,4.56,0.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2017,MI,Michigan,Wyoming,City,BRFSS,Health Outcomes,2688940,Chronic obstructive pulmonary disease among ad...,%,...,42.8910,-85.7067,280.38,0.00,8.31,80.11,0.41,5.63,7.18,6.14
496,2017,WA,Washington,Yakima,City,BRFSS,Health Outcomes,5380010,Chronic obstructive pulmonary disease among ad...,%,...,46.5926,-120.5478,211.95,0.07,0.81,117.30,0.60,4.04,4.58,0.81
497,2017,NY,New York,Yonkers,City,BRFSS,Health Outcomes,3684000,Chronic obstructive pulmonary disease among ad...,%,...,40.9465,-73.8637,293.73,0.00,23.99,90.84,5.60,6.75,9.18,2.19
498,2017,OH,Ohio,Youngstown,City,BRFSS,Health Outcomes,3988000,Chronic obstructive pulmonary disease among ad...,%,...,41.0993,-80.6463,250.34,0.00,4.28,92.98,1.30,6.70,6.93,0.67


In [26]:
# merge the asthma data with the air pollution data
casthsma_air_pollution_df  = pd.concat([casthma_df, all_cities_air_pollution_df], axis=1)
casthsma_air_pollution_df 

,Year,StateAbbr,StateDesc,CityName,GeographicLevel,DataSource,Category,UniqueID,Measure,Data_Value_Unit,...,Latitude,Longitude,CO,NO,NO2,O3,SO2,PM2.5,PM10,NH3
0,2017,TX,Texas,Abilene,City,BRFSS,Health Outcomes,4801000,Current asthma among adults aged >=18 Years,%,...,32.4547,-99.7383,213.62,0.03,1.80,75.10,0.33,5.45,10.60,1.74
1,2017,OH,Ohio,Akron,City,BRFSS,Health Outcomes,3901000,Current asthma among adults aged >=18 Years,%,...,41.0801,-81.5219,247.00,0.00,4.71,88.69,1.09,7.38,8.31,0.98
2,2017,CA,California,Alameda,City,BRFSS,Health Outcomes,600562,Current asthma among adults aged >=18 Years,%,...,37.7700,-122.2700,247.00,0.19,3.90,98.71,1.04,0.54,0.78,0.35
3,2017,GA,Georgia,Albany,City,BRFSS,Health Outcomes,1301052,Current asthma among adults aged >=18 Years,%,...,31.5776,-84.1763,230.31,0.00,2.91,80.82,0.36,5.88,8.28,2.34
4,2017,NY,New York,Albany,City,BRFSS,Health Outcomes,3601000,Current asthma among adults aged >=18 Years,%,...,42.6664,-73.7987,310.42,0.00,16.97,42.56,1.18,3.78,4.56,0.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2017,MI,Michigan,Wyoming,City,BRFSS,Health Outcomes,2688940,Current asthma among adults aged >=18 Years,%,...,42.8910,-85.7067,280.38,0.00,8.31,80.11,0.41,5.63,7.18,6.14
496,2017,WA,Washington,Yakima,City,BRFSS,Health Outcomes,5380010,Current asthma among adults aged >=18 Years,%,...,46.5926,-120.5478,211.95,0.07,0.81,117.30,0.60,4.04,4.58,0.81
497,2017,NY,New York,Yonkers,City,BRFSS,Health Outcomes,3684000,Current asthma among adults aged >=18 Years,%,...,40.9465,-73.8637,293.73,0.00,23.99,90.84,5.60,6.75,9.18,2.19
498,2017,OH,Ohio,Youngstown,City,BRFSS,Health Outcomes,3988000,Current asthma among adults aged >=18 Years,%,...,41.0993,-80.6463,250.34,0.00,4.28,92.98,1.30,6.70,6.93,0.67


In [27]:
# merge the chd data with the air pollution data
chd_air_pollution_df  = pd.concat([chd_df, all_cities_air_pollution_df], axis=1)
chd_air_pollution_df 

,Year,StateAbbr,StateDesc,CityName,GeographicLevel,DataSource,Category,UniqueID,Measure,Data_Value_Unit,...,Latitude,Longitude,CO,NO,NO2,O3,SO2,PM2.5,PM10,NH3
0,2017,TX,Texas,Abilene,City,BRFSS,Health Outcomes,4801000,Coronary heart disease among adults aged >=18 ...,%,...,32.4547,-99.7383,213.62,0.03,1.80,75.10,0.33,5.45,10.60,1.74
1,2017,OH,Ohio,Akron,City,BRFSS,Health Outcomes,3901000,Coronary heart disease among adults aged >=18 ...,%,...,41.0801,-81.5219,247.00,0.00,4.71,88.69,1.09,7.38,8.31,0.98
2,2017,CA,California,Alameda,City,BRFSS,Health Outcomes,600562,Coronary heart disease among adults aged >=18 ...,%,...,37.7700,-122.2700,247.00,0.19,3.90,98.71,1.04,0.54,0.78,0.35
3,2017,GA,Georgia,Albany,City,BRFSS,Health Outcomes,1301052,Coronary heart disease among adults aged >=18 ...,%,...,31.5776,-84.1763,230.31,0.00,2.91,80.82,0.36,5.88,8.28,2.34
4,2017,NY,New York,Albany,City,BRFSS,Health Outcomes,3601000,Coronary heart disease among adults aged >=18 ...,%,...,42.6664,-73.7987,310.42,0.00,16.97,42.56,1.18,3.78,4.56,0.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2017,MI,Michigan,Wyoming,City,BRFSS,Health Outcomes,2688940,Coronary heart disease among adults aged >=18 ...,%,...,42.8910,-85.7067,280.38,0.00,8.31,80.11,0.41,5.63,7.18,6.14
496,2017,WA,Washington,Yakima,City,BRFSS,Health Outcomes,5380010,Coronary heart disease among adults aged >=18 ...,%,...,46.5926,-120.5478,211.95,0.07,0.81,117.30,0.60,4.04,4.58,0.81
497,2017,NY,New York,Yonkers,City,BRFSS,Health Outcomes,3684000,Coronary heart disease among adults aged >=18 ...,%,...,40.9465,-73.8637,293.73,0.00,23.99,90.84,5.60,6.75,9.18,2.19
498,2017,OH,Ohio,Youngstown,City,BRFSS,Health Outcomes,3988000,Coronary heart disease among adults aged >=18 ...,%,...,41.0993,-80.6463,250.34,0.00,4.28,92.98,1.30,6.70,6.93,0.67
